# **Measure Organelle Interactions**

***Prior to this notebook, you should have already run through [2.0_quantification_setup](2.0_quantification_setup.ipynb).***

The `'methods_...'` notebooks included here in infer-subc Part 2: quantification will go over how each of the quantification methods (morphology, interactions, and distribution) are carried out. The notebooks will explain each step in the method and display the combined function at the end of the notebook. 

### **Biological Relevance**
Intracellular organelles do not exist independently of one another. In recent years, the existance and function of organelle contact site, regions of close apposition between membrane bound organelles has been recognized. They facilitate protein, lipid, and metabolite transport, coordinate organelle trafficking and function, and are involved in many cellular pathways and functions [[1](https://www.cell.com/cell/pdf/S0092-8674(23)01328-4.pdf)]. 

The distance between membranes at contact sites is between 30-80 nm depending on the types of organelles involved. This distance is not resolvable using standard confocal microscopy images [[2](https://zeiss-campus.magnet.fsu.edu/articles/basics/resolution.html)]. However, interactions between organelle which can include organelle contact sites can be estimated through overlap in label localization in confocal microscopy images.

### **Oragnelle Interactions** 📐
Here, we will use the overlapping area between two or more organelle objects as putative interaction sites. The current notebook is only formatted to collect information about pairwise interactions between organelles, but this will be expanded in future versions to include all possible n-way interaction types.

**`Organelle interaction sites`**: regions of overlap between two or more organelles

These sites can then be measured for features such as number, size, and shape utilizing the `get_morpholgy_metrics()` function outlined in [method_morphology](method_morphology.ipynb) notebook and/or measurements of subcellular distribution utilizing the `get_distribution()` function outlined in [method_distribution](method_distribution.ipynb) notebook.

-----

### 👣 **Summary of steps**  

🛠️ **BUILD FUNCTION STEP-BY-STEP**

- **`STEP 1`** - Select two organelles of interest

- **`STEP 2`** - Create pairwise interaction sites

- **`STEP 3`** - Apply mask to the object of interest

- **`STEP 4`** - Measure interaction site morphology

- **`STEP 5`** - Find the ID number of the organelles involved in each interaction site

- **`STEP 6`** - Measure interaction site distribution

⚙️ **DEFINE AND TEST *`Interactions`* FUNCTION**

- Define `get_interactions_metrics` function
- Run `get_interactions_metrics` function

The above steps will be applied to pair of organelles of interest. Batch processing is available in a separate notebook.

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`2.0_quantification_setup`](2.0_quantification_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

In [ ]:
from pathlib import Path
import os
import warnings

import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.measure import (regionprops, regionprops_table)

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_assert_uint16_labels)
from infer_subc.utils.stats_helpers import *
from infer_subc.organelles import * 

%load_ext autoreload
%autoreload 2

#### &#x1F3C3; **Run code; no user input required**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `raw_img_type`, `data_root_path`, `raw_data_path`, `seg_data_path`, and `quant_data_path`.

In [ ]:
#### USER INPUT REQUIRED ###
raw_img_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
raw_data_path = data_root_path / "raw_single"
seg_data_path = data_root_path / "out_single"
quant_data_path = data_root_path / "quant_single"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(quant_data_path):
    Path.mkdir(quant_data_path)
    print(f"making {quant_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
raw_img_file_list = list_image_files(raw_data_path,raw_img_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [ ]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = raw_img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']

print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi
Channel 0 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


#### &#x1F6D1; &#x270D; **User Input Required:**

Specify the following information about the segmentation files: - `org_file_names`, `org_channels_ordered`, `regions_file_names`, `suffix_separator`, and `mask_name`.

In [ ]:
#### USER INPUT REQUIRED ###
org_file_names = ["lyso", "mito", "golgi", "perox", "ER", "LD"]
org_channels_ordered = [1, 2, 3, 4, 5, 6]
regions_file_names = ["cell", "nuc"]
suffix_separator = "-"
mask_name = "cell"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# find file paths for segmentations
all_suffixes = org_file_names + regions_file_names
filez = find_segmentation_tiff_files(file_path, all_suffixes, seg_data_path, suffix_separator)

# read the segmentation and masks/regions files into memory
organelles = [read_tiff_image(filez[org]) for org in org_file_names]
regions = [] 
for m in regions_file_names:
    mfile = read_tiff_image(filez[m])
    regions.append(mfile)

# match the intensity channels to the segmentation files
intensities = [img_data[ch] for ch in org_channels_ordered]

# specifiy the mask image
m = regions_file_names.index(mask_name)
mask = regions[m]

# open viewer and add images
viewer = napari.Viewer()
for r, reg in enumerate(regions_file_names):
    viewer.add_image(regions[r],
                     scale=scale,
                     name=f"{reg} mask")

for o, org in enumerate(org_file_names):
    viewer.add_image(intensities[o],
                     scale=scale,
                     name=f"{org} intensity channel")
    viewer.add_labels(organelles[o],
                      scale=scale,
                      name=f"{org} segmentation")
viewer.grid.enabled = True
viewer.reset_view()

print("The following matching files were found and can now be viewed in Napari:")
filez


09-Feb-25 13:14:41 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x661+1530+213 (frame: 1104x708+1521+175) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 905x831+1528+206 (frame: 923x878+1519+168) margins: 9, 38, 9, 9 minimum size: 385x519 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=403,566 maxtrack=0,0)


The following matching files were found and can now be viewed in Napari:


{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw_single/a24hrs_Ctrl_14_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-LD.tiff'),
 'cell': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-cell.tiff'),
 'nuc': WindowsPath('C:/Use

-----
## **QUANTIFY INTERACTIONS BETWEEN *two organelles* FROM <INS>ONE CELL</INS>**

### **`STEP 1` - Select two organelles of interest**

&#x1F453; **FYI:** To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask to the segmentation file. This will exclude any objects outside of the mask area from the analysis.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify which organelle you would like to examine in this analysis and which segmentation file should be used as the mask:
- `org_a_name`: the suffix of the first organelle you would like to measure interactions for; it should match one of the names included in the "org_file_names" variable above
- `org_b_name`: the suffix of the second organelle you would like to measure interactions for; it should match one of the names included in the "org_file_names" variable above

In [ ]:
#### USER INPUT REQUIRED ###
org_a_name = "lyso"
org_b_name = "mito"

### **`STEP 2` - Create pairwise interaction sites**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This block of code selects the voxels within the image that are shared between the two organelles using a `logical OR` opporation. The output from this step is included in the Napari window. The interaction sites are highlighted in white and the two organelles are colors magenta and green.

In [ ]:
# ensure the organelles chosen are labeled with ID numbers
a = _assert_uint16_labels(org_a_name)
b = _assert_uint16_labels(org_b_name)

# logical OR
a_int_b = np.logical_and(a > 0, b > 0)

# visualize output in Napari
viewer.layers.clear()
viewer.add_image(a>0, colormap='green', blending ='additive')
viewer.add_image(b>0, colormap='magenta', blending ='additive')
viewer.add_image(a_int_b>0, blending ='additive')
nbscreenshot(viewer, canvas_only=True)

### **`STEP 3` - Measure interaction site morphology**

In [ ]:
# TODO: figure out how to access the props table so that the interaction IDs can be created as done below

aXb_interaction_tab = get_morphology_metrics(segmentation_img=a_int_b, 
                                            seg_name=f"{org_a_name}X{org_b_name}", 
                                            intensity_img=None, 
                                            mask=mask, 
                                            mask_name=mask_name,
                                            scale=scale)

### **`STEP 5` - Find the ID number of the organelles involved in each interaction site**

In [ ]:
label_a = []
index_ab = []
label_b = []
for index, lab in enumerate(props["label"]):
    # this seems less elegant than you might wish, given that regionprops returns a slice,
    # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
    volume = labels[props["slice"][index]]
    la = a[props["slice"][index]]
    lb = b[props["slice"][index]]
    volume = volume == lab
    la = la[volume]
    lb = lb[volume]

    all_as = np.unique(la[la>0]).tolist()
    all_bs = np.unique(lb[lb>0]).tolist()
    if len(all_as) != 1:
        print(f"we have an error.  as-> {all_as}")
    if len(all_bs) != 1:
        print(f"we have an error.  bs-> {all_bs}")

    label_a.append(f"{all_as[0]}" )
    label_b.append(f"{all_bs[0]}" )
    index_ab.append(f"{all_as[0]}_{all_bs[0]}")

### **`STEP 6` - Measure interaction site distribution**

### **`DEFINE` - `get_interaction_metrics` function**